In [1]:
pattern = '*1.tab'

In [2]:
import pandas as pd
import geopandas as gpd
import mio
import numpy as np
from matplotlib import pyplot as plt
import pathlib
import time

In [3]:
def tri_array(p0, p1, p2, area, px, py):
    
    r_shape = tri_shape(p0, p1, p2, area, px, py)
    r_heights = tri_heights(p0, p1, p2, area, px, py)
    
    return r_shape * r_heights

In [4]:
def tri_heights(p0, p1, p2, area, px, py):
    # calculate the triangle plane
    Ax, Ay, Az = p0
    Bx, By, Bz = p1
    Cx, Cy, Cz = p2
    
    AB = (Bx - Ax, By - Ay, Bz - Az)
    AC = (Cx - Ax, Cy - Ay, Cz - Az)
    
    # AB x AC = (a,b,c)
    # Plane equasion: ax + by + cz + d = 0
    a, b, c = np.cross(AB, AC)
    d = -(a * Ax + b * Ay + c * Az)
 
    pz  = (-a * px - b * py - d) / c

    return pz

In [5]:
def tri_shape(p0, p1, p2, area, px, py):
    # calc the outline of the triangles
    p0x, p0y, _ = p0
    p1x, p1y, _ = p1
    p2x, p2y, _ = p2
    
    # do the calc, using stolen formula
    s = 1/(2*area)*(p0y*p2x - p0x*p2y + (p2y - p0y)*px + (p0x - p2x)*py)
    t = 1/(2*area)*(p0x*p1y - p0y*p1x + (p0y - p1y)*px + (p1x - p0x)*py)
    u = 1 - s -t
    r = (s>0) & (t>0) & (u>0)
    
    return r.astype(int)

In [6]:
def signed_area(p0, p1, p2):
        # calc signed area of triangle
        p0x, p0y, _ = p0
        p1x, p1y, _ = p1
        p2x, p2y, _ = p2
        
        a = 0.5 *(-p1y*p2x + p0y*(-p1x + p2x) + p0x*(p1y - p2y) + p1x*p2y)

        return a

In [7]:
def create_meshgrid(p0, p1, p2):
    xmin = int(min(p0[0], p1[0], p2[0])) -1
    xmax = int(max(p0[0], p1[0], p2[0])) +1
    ymin = int(min(p0[1], p1[1], p2[1])) -1
    ymax = int(max(p0[1], p1[1], p2[1])) +1
    df = pd.DataFrame(
        columns = range(xmin, xmax, 1),
        index = range(ymax, ymin, -1),
        data = 0
    )
    xa = np.arange(xmin, xmax,  1)
    ya = np.arange(ymax, ymin, -1)
    px, py = np.meshgrid(xa, ya)
    return xa, ya, px, py

In [8]:
def get_df_list(tab):
    print(f'reading {tab}', end=' ')
    vec = gpd.read_file(tab)
    ldf = []
    count_ok = 0
    count_er = 0
    
    i = 0
    print(len(vec) / 1e3, 'x 1000 triangles')
    print('create mini rasters')
    for ind, row in vec.iterrows():
        i += 1
        mio.show_perc(i, len(vec), 10000)
        p0 = (row.x0, row.y0, row.z0)
        p1 = (row.x1, row.y1, row.z1)
        p2 = (row.x2, row.y2, row.z2)

        xa, ya, px, py = create_meshgrid(p0, p1, p2)

        area = signed_area(p0, p1, p2)

        # get rid of mini areas
        if area >= 0.5:
            count_ok += 1
            ar = tri_array(p0, p1, p2, area, px, py)
            df = pd.DataFrame(ar, columns=xa, index=ya)
            if len(df) > 0:
                ldf.append(df)
        else:
            # propably vertical
            count_er += 1
    return ldf

In [9]:
def get_big_empty(ldf):
    lx, ly = [], []
    for df in ldf:
        lx.append(df.columns.min())
        lx.append(df.columns.max())
        ly.append(df.index.min())
        ly.append(df.index.max())
    xmin, xmax = min(lx)-5, max(lx)+5
    ymin, ymax = min(ly)-5, max(ly)+5
    
    empty = pd.DataFrame(
        columns = range(xmin, xmax, 1),
        index = range(ymax, ymin, -1),
        data = 0
    )
    return empty

In [10]:
def get_tag_todo():
    source_dir = pathlib.Path('../sb_tab')
    tags_all = set([d.stem for d in source_dir.glob(pattern)])

    dest_dir = pathlib.Path('../sb_tif')
    tags_done =  set([t.stem[0:7] for t in dest_dir.glob('*.tif')])
    #print(tags_done)

    tags_todo = tags_all.difference(tags_done)
    #print(tags_todo)
    l = list(tags_todo)
    l.sort()
    if len(l) >0:
        print(f"{len(l)} files to do")
        return l[0]
    else:
        return

In [11]:
get_tag_todo()

618 files to do


'1129-21'

In [ ]:
# MAIN
while True:
    tag = get_tag_todo()
    if tag is not None:
        # Create list of mini dataframes
        ldf = get_df_list(f'../sb_tab/{tag}.tab')

        # Create empty raster
        big = get_big_empty(ldf)

        print('\ncombining all the mini rasters')
        for i, small in enumerate(ldf):
            mio.show_perc(i, len(ldf), 2_000)
            xmin, xmax = small.columns.min(), small.columns.max()
            ymin, ymax = small.index.min(), small.index.max()
            big.loc[ymax:ymin, xmin:xmax] = np.maximum(big.loc[ymax:ymin, xmin:xmax], small)

        # Save raster file
        print(f'writing {tag}_raster.tif')
        res = (big.fillna(0) + 0.5).astype('int16')
        mio.write_raster(res, f'../sb_tif/{tag}_raster.tif')
        time.sleep(10)

618 files to do
reading ../sb_tab/1129-21.tab 96.857 x 1000 triangles
create mini rasters
10.32% 20.65% 30.97% 41.3% 51.62% 61.95% 72.27% 82.6% 92.92% 
combining all the mini rasters
0.0% 11.26% 22.51% 33.77% 45.03% 56.28% 67.54% 78.8% 90.05% writing 1129-21_raster.tif
617 files to do
reading ../sb_tab/1129-31.tab 55.207 x 1000 triangles
create mini rasters
18.11% 36.23% 54.34% 72.45% 90.57% 
combining all the mini rasters
0.0% 19.62% 39.25% 58.87% 78.49% 98.12% writing 1129-31_raster.tif
616 files to do
reading ../sb_tab/1129-41.tab 46.987 x 1000 triangles
create mini rasters
21.28% 42.56% 63.85% 85.13% 
combining all the mini rasters
0.0% 17.2% 34.4% 51.6% 68.8% 86.0% writing 1129-41_raster.tif
615 files to do
reading ../sb_tab/1130-11.tab 100.311 x 1000 triangles
create mini rasters
9.97% 19.94% 29.91% 39.88% 49.84% 59.81% 69.78% 79.75% 89.72% 99.69% 
combining all the mini rasters
0.0% 11.5% 23.0% 34.5% 46.0% 57.5% 69.0% 80.5% 92.0% writing 1130-11_raster.tif
614 files to do
readin